In [ ]:
from openapi_client import openapi
from datetime import datetime, timedelta
from pytz import timezone
import pandas as pd
import time
from dotenv import load_dotenv, find_dotenv
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)
token = os.environ.get('APIKEY_SANDBOX')
 
client = openapi.sandbox_api_client(token)
client.sandbox.sandbox_register_post()
client.sandbox.sandbox_clear_post()
client.sandbox.sandbox_currencies_balance_post(sandbox_set_currency_balance_request={"currency": "USD", "balance": 1000})

In [ ]:
def get_instruments(instrument=None):
    """
    Gets list of instruments (bonds, stocks, ETFs that traded on tinkoff-invest broker)

    Args:
        instrument(str) : can be either stock, bond, etf. 
   
    Returns:
        df (Pandas dataframe) : list of instruments available for trade

    Raises:
        Error if not defined instrument or wrong instrument name is provided as input.
    """
    if instrument=='stock':
        tickers=client.market.market_stocks_get_with_http_info()
    elif instrument=='bond':
        tickers=client.market.market_bonds_get_with_http_info()
    elif instrument=='etf':
        tickers=client.market.market_etfs_get_with_http_info()
    else:
        print(f'Wrong instrument defined. Acceptable are stock, bond, etf. You provided {instrument}')
        return None
    list_of_tickers=[]
    for item in tickers[0].payload.instruments:
        list_of_tickers.append(item.to_dict())
    df=pd.DataFrame.from_records(list_of_tickers)
    return df

In [ ]:
tick=get_instruments(instrument='stock')

In [ ]:
tick

In [ ]:
bond=get_instruments(instrument='bond')

In [ ]:
bond

In [ ]:
etf=get_instruments(instrument='etf')


In [ ]:
etf

In [ ]:
def detailed_history(figi='BBG00M0C8YM7',
                    _from=None,
                    to=None,
                    interval="1min"):
    if to==None:
        endTime=datetime.now()
    else:
        endTime= to
    if _from==None:
        days_span=1
        startTime = datetime.now() - timedelta(days=days_span)
    else:
        startTime=_from
    #print(endTime)
    #print(startTime)
    
    startTime_list=[]
    endTime_list=[]
    
    startTime_list.append(startTime.strftime('%Y-%m-%dT%H:%M:%S')+"+07:00")
    endTime_list.append(endTime.strftime('%Y-%m-%dT%H:%M:%S')+"+07:00")
    
    if (endTime-startTime).days>1:
        startTime_list=[]
        endTime_list=[]
        endTime=to
        for i in range((to-_from).days):
            #print(i)
            newStartTime=endTime-timedelta(days=1)
            startTime_list.append(newStartTime.strftime('%Y-%m-%dT%H:%M:%S')+"+07:00")
            endTime_list.append(endTime.strftime('%Y-%m-%dT%H:%M:%S')+"+07:00")
            endTime=newStartTime
    #print(startTime_list)
    #print(endTime_list)
    list_df=[]
    for i in range(len(startTime_list)):
        tickers=client.market.market_candles_get_with_http_info(figi=figi,
                                                        _from=startTime_list[i],
                                                        to=endTime_list[i],
                                                        interval="1min")
        list_of_tickers=[]
        for item in tickers[0].payload.candles:
            list_of_tickers.append(item.to_dict())
        df=pd.DataFrame.from_records(list_of_tickers)
        list_df.append(df)
        
    df_merge = list_df[0]
    n=len(list_df)
    print(f"Concat frame 0. Dataframe with data (row,col) : {df_merge.shape}.")
    for i in range(1, n):
        df_merge = pd.concat([df_merge, list_df[i]], sort=False)
        print(f"Concat frame {i}. Dataframe with data (row,col) : {df_merge.shape}.")
    return df_merge
            
        
    
    

In [ ]:
#test#1 - 1 day request
from datetime import datetime
days_span=1
startTime = datetime.now() - timedelta(days=days_span)
endTime = datetime.now()
print("test1")
df_merge1=detailed_history(_from=startTime,
                    to=endTime)
#test#2 - 10 days request
from datetime import datetime
days_span=10
startTime = datetime.now() - timedelta(days=days_span)
endTime = datetime.now()
print("test2")
df_merge2=detailed_history(_from=startTime,
                    to=endTime)
#test#3-no data provided
print("test3")
df_merge3=detailed_history()

In [ ]:
df_merge2

In [ ]:
df_merge2.info()

In [ ]:
from datetime import datetime
days_span=10
startTime = datetime.now() - timedelta(days=days_span)
endTime = datetime.now()
for figi in etf.figi.unique():
    print("retrieving figi: "+figi+ " ticker: "+etf[etf.figi==figi].ticker.values)
    list_df=[]
    df=detailed_history(figi=figi,
                                                        _from=startTime,
                                                        to=endTime,
                                                        interval="1min")
    time.sleep(5)
    list_df.append(df)
df_merge = list_df[0]
n=len(list_df)
print(f"Concat frame 0. Dataframe with data (row,col) : {df_merge.shape}.")
for i in range(1, n):
    df_merge = pd.concat([df_merge, list_df[i]], sort=False)
    print(f"Concat frame {i}. Dataframe with data (row,col) : {df_merge.shape}.")



In [ ]:
from tqdm import tqdm
def get_detailed_data(data,_from=None,to=None):
    
    for figi in tqdm(data.figi.unique()):
        print("retrieving figi: "+figi+ " ticker: "+data[data.figi==figi].ticker.values)
        list_df=[]
        df=detailed_history(figi=figi,
                                                            _from=_from,
                                                            to=to,
                                                            interval="1min")
        time.sleep(3)
        list_df.append(df)
    
    df_merge = list_df[0]
    n=len(list_df)
    print(f"Concat frame 0. Dataframe with data (row,col) : {df_merge.shape}.")
    for i in range(1, n):
        df_merge = pd.concat([df_merge, list_df[i]], sort=False)
        print(f"Concat frame {i}. Dataframe with data (row,col) : {df_merge.shape}.")
    return df_merge

In [ ]:
from datetime import datetime
days_span=90
startTime = datetime.now() - timedelta(days=days_span)
endTime = datetime.now()
tick_data=get_detailed_data(tick,_from=startTime,to=endTime)

In [ ]:
etf_data=get_detailed_data(etf,_from=startTime,to=endTime)

In [ ]:
bonds_data=get_detailed_data(bond,_from=startTime,to=endTime)

In [ ]:
def set_balance():
    balance_set = client.sandbox.sandbox_currencies_balance_post({"currency": "USD", "balance": 10000})
    print("balance")
    print(balance_set)
    print()


def print_24hr_operations():
    now = datetime.now(tz=timezone('Europe/Moscow'))
    yesterday = now - timedelta(days=1)
    ops = client.operations.operations_get(_from=yesterday.isoformat(), to=now.isoformat())
    print("operations")
    print(ops)
    print()


def print_orders():
    orders = client.orders.orders_get()
    print("active orders")
    print(orders)
    print()


def make_order():
    order_response = client.orders.orders_limit_order_post(figi='BBG009S39JX6',
                                                           limit_order_request={"lots": 1,
                                                                                "operation": "Buy",
                                                                                "price": 0.01})
    print("make order")
    print(order_response)
    print()
    return order_response


# won't work in sandbox - orders are being instantly executed
def cancel_order(order_id):
    cancellation_result = client.orders.orders_cancel_post(order_id=order_id)
    print("cancel order")
    print(cancellation_result)
    print()


set_balance()
print_24hr_operations()
print_orders()
order_response = make_order()
print_orders()
# cancel_order(order_response.payload.order_id)
# print_orders()